*Last update: 23.08.2022*  
*Contact : fdrs@ifrc.org or simon.weiss@ifrc.org*  
*FDRS focal point: Simon Weiss, FDRS Data Analyst*    

# How to deal with Missing Values? FDRS imputation methodology in python
-----------


The FDRS is ambitious and wide reaching. Although the data quality and reporting are improving each year, data is missing for some National Societies. As a result, some data fluctuations may be misleading: trend lines can drop for a given year when there is missing data, and some National Societies are excluded from the total and then appear again in another year.In order to better represent the network and better count everyone, FDRS implement every year data imputation techniques. 

The purpose of this notebook is to apply the method selected by the FDRS team and to interact with the FDRS backoffice with a post method to publish the imputed values. 
The ingested data is  replicated and displayed on the website https://data.ifrc.org/FDRS/ and used in FDRS research such as [Everyone count report](https://data-api.ifrc.org/documents/noiso/Everyone%20Counts%20Report%202022%20EN.pdf). 

Therefore this is not a research notebook but a production notebook. It aims to simply expose the FDRS methodology of imputation step by step. A related python script has been created: [imputing.py]() which allows to launch this methodology once. 

The approach chosen was to replace the 2019, 2020 and 2021 missing data as well as to apply two different techniques according to the indicator categories, in the previous years all NSs reported their data then no input technique was employed.   

The imputing applies only to main indicators and does not apply to disaggregated levels to maintain consistency across years. A detailed description of the methodology is available in the pdf [Missing Data]()


This notebook contains four sections

 - [Get FDRS Data](#getdata)
 - [Run imputing missing values](#imputing)
 - [Optimization (optional)](#opti)
 - [Post method - interact FDRS BE](#post)
 
We'll get started by loading the python modules in our analysis toolkit.

## 0.Import package

In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import requests, json
from pandas import json_normalize
import json
from datetime import datetime
import os
import sys
import warnings
import time
warnings.filterwarnings('ignore')


<a id="getdata"></a>
## 1. Get Data

---------------


### 1.1 Import modules

In [2]:
#import api functions to get data
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\src\data")
from fdrsapi import api_function,baseline, api_function_imputed


In [3]:
#generate fdrs codebook and store it in references folder
KPI = requests.get(f"https://data-api.ifrc.org/api/indicator?apiKey=21e401ae-6b35-404b-a72a-b74cce66dee3").json()
KPI = json_normalize(data=KPI)
#export KPI to excel in references folder
KPI.to_excel(r"..\references\codebook.xlsx")

In [4]:
#import codebook 
codebook = pd.read_excel(r"..\references\codebook.xlsx")
#select only  NS Reach CPD KPIs
kpi=(codebook.query('KPI_Note== "NS Reach - CPD"')["KPI_Code"]).to_list()
#add manualy the other KPIS
kpi_code=kpi+["KPI_TrainFA_Tot","KPI_DonBlood_Tot"]+["KPI_GB_Tot","KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_noLocalUnits","KPI_IncomeLC_CHF" ,"KPI_expenditureLC_CHF"]+["validated"]+["KPI_WasSubmitted"]

In [5]:
#counting nb of considered KPIS + removing metadata kpis 
print("number of total KPI considered:", len(kpi_code)-2)

number of total KPI considered: 23


New indicators: remove

In [6]:
#remove the 2021 new indicators KPI collected from kpi_code
old_kpi=kpi_code.copy()
new_kpi=["KPI_Climate_CPD", "KPI_ClimateHeat_CPD", "KPI_ReachHI_CPD", "KPI_ReachHPM_CPD","KPI_ReachRCRCEd_CPD"]
for x in new_kpi:
    old_kpi.remove(x)

In [7]:
#check  final kpi list
old_kpi
print("number of old KPI considered:", len(old_kpi))

number of old KPI considered: 20


### 1.2 Get data from FDRS API

In [8]:
#read public api key from config file
f = open(module_path+"/src/config/public_api_key.txt")
api_key=f.readline()
#select years to download data from backoffice
years=["2018","2019","2020","2021"]
kpi_float=old_kpi.copy()
#run api function for each year
time_series=api_function(years,old_kpi,kpi_float,api_key)
time_series

2018
2019
2020
2021
number of KPI downloaded: 20


,KPI ID,KPI_DON_code,NSO_DON_name,NSO_ZON_name,iso_3,KPI_Year,KPI_DonBlood_Tot,KPI_GB_Tot,KPI_IncomeLC_CHF,KPI_PStaff_Tot,...,KPI_ReachL_CPD,KPI_ReachM_CPD,KPI_ReachSI_CPD,KPI_ReachS_CPD,KPI_ReachWASH_CPD,KPI_TrainFA_Tot,KPI_WasSubmitted,KPI_expenditureLC_CHF,KPI_noLocalUnits,validated
0,DAF0012018,DAF001,Afghan Red Crescent,Asia Pacific,AFG,2018,0.0,15.0,1.803370e+07,2195.0,...,NaN,NaN,NaN,NaN,NaN,1020.0,1.0,1.847249e+07,122.0,1.0
1,DAL0012018,DAL001,Albanian Red Cross,Europe and Central Asia,ALB,2018,6531.0,14.0,1.555610e+06,53.0,...,12800.0,0.0,34150.0,0.0,47000.0,62709.0,1.0,1.348786e+06,39.0,1.0
2,DDZ0012018,DDZ001,Algerian Red Crescent,Middle East and North Africa,DZA,2018,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,248.0,1.0
3,DUS0012018,DUS001,American Red Cross,Americas,USA,2018,2662986.0,15.0,3.656908e+09,19649.0,...,2308469.0,0.0,0.0,0.0,0.0,2283480.0,1.0,3.193028e+09,263.0,1.0
4,DAD0012018,DAD001,Andorran Red Cross,Europe and Central Asia,AND,2018,1084.0,9.0,1.177623e+06,20.0,...,NaN,NaN,1117.0,NaN,NaN,2901.0,1.0,1.379745e+06,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,DVE0012021,DVE001,Venezuelan Red Cross,Americas,VEN,2021,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
764,DVN0012021,DVN001,Vietnam Red Cross Society,Asia Pacific,VNM,2021,1304191.0,5.0,NaN,16008.0,...,40545.0,NaN,NaN,16679.0,256570.0,26811.0,1.0,NaN,12693.0,NaN
765,DYE0012021,DYE001,Yemen Red Crescent Society,Middle East and North Africa,YEM,2021,NaN,6.0,6.513051e+06,454.0,...,538645.0,576219.0,21534.0,99003.0,419470.0,3404.0,1.0,5.273168e+06,32.0,1.0
766,DZM0012021,DZM001,Zambia Red Cross Society,Africa,ZMB,2021,NaN,10.0,2.930602e+06,66.0,...,1249.0,4807.0,NaN,169.0,238278.0,1720.0,1.0,2.896418e+06,59.0,1.0


#### 1.3 Deal with 2021 New Indicators

**Deal with new indicators**: <!> To update everytime we run the script
- No values at Kpivalue? for the 2021 FDRS new values, only available at data? endpoint
- From Powerbi pipeline set up a excel export called "FDRS2021NewKPIValues"
- Applied some power query transformation (KPI names reverted back to KPI code, selected Year)
- Moved it in the fdrs_imputing_missing_values repo
- Perfomed Merge Statement in pandas Left join on iso and years
- time_series completed with new value 



In [9]:
#read excel from data/raw folder
new_time_series=pd.read_excel(module_path+"/data/raw/FDRS2021NewKPIValues.xlsx")

In [10]:
#print new_time_series to check
new_time_series

,iso_3,KPI_Year,KPI_ReachRCRCEd_CPD,KPI_Climate_CPD,KPI_ClimateHeat_CPD,KPI_ReachHI_CPD,KPI_ReachHPM_CPD
0,AFG,2018,NaN,NaN,NaN,NaN,NaN
1,AFG,2019,NaN,NaN,NaN,NaN,NaN
2,AFG,2020,NaN,NaN,NaN,NaN,NaN
3,AFG,2021,105.0,NaN,NaN,455730.0,4646.0
4,AGO,2018,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
681,ZMB,2021,0.0,0.0,NaN,0.0,382.0
682,ZWE,2018,NaN,NaN,NaN,NaN,NaN
683,ZWE,2019,NaN,NaN,NaN,NaN,NaN
684,ZWE,2020,NaN,NaN,NaN,NaN,NaN


In [11]:
#merge new time series with old time series
time_series=time_series.merge(new_time_series, how='left', on=['iso_3','KPI_Year'])

In [12]:
#print to check
time_series

,KPI ID,KPI_DON_code,NSO_DON_name,NSO_ZON_name,iso_3,KPI_Year,KPI_DonBlood_Tot,KPI_GB_Tot,KPI_IncomeLC_CHF,KPI_PStaff_Tot,...,KPI_TrainFA_Tot,KPI_WasSubmitted,KPI_expenditureLC_CHF,KPI_noLocalUnits,validated,KPI_ReachRCRCEd_CPD,KPI_Climate_CPD,KPI_ClimateHeat_CPD,KPI_ReachHI_CPD,KPI_ReachHPM_CPD
0,DAF0012018,DAF001,Afghan Red Crescent,Asia Pacific,AFG,2018,0.0,15.0,1.803370e+07,2195.0,...,1020.0,1.0,1.847249e+07,122.0,1.0,NaN,NaN,NaN,NaN,NaN
1,DAL0012018,DAL001,Albanian Red Cross,Europe and Central Asia,ALB,2018,6531.0,14.0,1.555610e+06,53.0,...,62709.0,1.0,1.348786e+06,39.0,1.0,NaN,NaN,NaN,NaN,NaN
2,DDZ0012018,DDZ001,Algerian Red Crescent,Middle East and North Africa,DZA,2018,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,248.0,1.0,NaN,NaN,NaN,NaN,NaN
3,DUS0012018,DUS001,American Red Cross,Americas,USA,2018,2662986.0,15.0,3.656908e+09,19649.0,...,2283480.0,1.0,3.193028e+09,263.0,1.0,NaN,NaN,NaN,NaN,NaN
4,DAD0012018,DAD001,Andorran Red Cross,Europe and Central Asia,AND,2018,1084.0,9.0,1.177623e+06,20.0,...,2901.0,1.0,1.379745e+06,1.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,DVE0012021,DVE001,Venezuelan Red Cross,Americas,VEN,2021,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
764,DVN0012021,DVN001,Vietnam Red Cross Society,Asia Pacific,VNM,2021,1304191.0,5.0,NaN,16008.0,...,26811.0,1.0,NaN,12693.0,NaN,NaN,NaN,0.0,NaN,NaN
765,DYE0012021,DYE001,Yemen Red Crescent Society,Middle East and North Africa,YEM,2021,NaN,6.0,6.513051e+06,454.0,...,3404.0,1.0,5.273168e+06,32.0,1.0,18300.0,NaN,NaN,41879.0,1769102.0
766,DZM0012021,DZM001,Zambia Red Cross Society,Africa,ZMB,2021,NaN,10.0,2.930602e+06,66.0,...,1720.0,1.0,2.896418e+06,59.0,1.0,0.0,0.0,NaN,0.0,382.0


##### 1.4 Pre-processing for the imputation : validate previous years data

We Consider that previous year data has been validated

In [13]:
#count nb of validated submission per years
time_series.query('KPI_Year == 2019')["validated"].sum(),time_series.query('KPI_Year == 2020')["validated"].sum(), time_series.query('KPI_Year == 2021')["validated"].sum()

(158.0, 167.0, 48.0)

In [14]:
##replace validated =0 for KPI year 2020,2019,2018 with validated =1
time_series.loc[(time_series["KPI_Year"]==2020) & (time_series["validated"]==0), "validated"]=1
time_series.loc[(time_series["KPI_Year"]==2019) & (time_series["validated"]==0), "validated"]=1
time_series.loc[(time_series["KPI_Year"]==2018) & (time_series["validated"]==0), "validated"]=1



<a id="imputing"></a>
## 2. Run imputing missing values
---------------

### 2.1 Import modules

In [15]:
#import imputing function 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\src\data")
from imp_function import imputing_mean,imputing_na


# 2.2 Run imputing functions

In [16]:
data_clean=time_series.copy()


# 1st subset:before 2018
data_clean1=data_clean.query('KPI_Year >= 2012 & KPI_Year < 2018').copy()

# 2nd subset: after 2018 and validated
data_clean2=data_clean.query('KPI_Year >= 2018 & KPI_Year < 2022 & validated == 1').copy()



# 3rd subset: after 2018 and all data; idea : imputing all then filter out the validated data
data_clean3=data_clean.query('KPI_Year >= 2018').copy()
data_clean3.loc[(data_clean3["KPI_Year"]==2021) & (data_clean3["validated"]==0.0), kpi_code]=np.nan

# Imputing : NS Reach CPD KPIs
data_clean3_na_columns=data_clean3.copy()
#import codebook 
codebook = pd.read_excel(r"..\references\codebook.xlsx")
#select only NS Reach CPD KPIs
kpi=(codebook.query('KPI_Note== "NS Reach - CPD"')["KPI_Code"]).to_list()
#add manualy the other Reach KPIS
kpi_reach_code=kpi+["KPI_TrainFA_Tot","KPI_DonBlood_Tot"]
selected_cols=kpi_reach_code
imputing_mean(data_clean3_na_columns,selected_cols,data_clean3)


# Imputing : NS Finance and Parnership & NS Governance and Structure Section
kpi_gov_fi_code=["KPI_GB_Tot","KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_noLocalUnits","KPI_IncomeLC_CHF" ,"KPI_expenditureLC_CHF"]
selected_cols_2d=kpi_gov_fi_code.copy()

# 18.11.21Comment : After reviewing with the Team, "supported1" and "received_support1" have been imputed seperatly in the script FDRS Network Data Transformation
imputing_na(data_clean3_na_columns,selected_cols_2d)


data_clean3_na_columns=data_clean3_na_columns.query('KPI_Year >= 2018 & KPI_Year < 2022 & validated != 1').copy()

# concatenate the two sub_datasets
fdrs_na_columns = [data_clean1, data_clean2, data_clean3_na_columns]
fdrs_data_fdrs_na_columns = pd.concat(fdrs_na_columns)


KPI_Climate_CPD
KPI_ClimateHeat_CPD
KPI_ReachCTP_CPD
KPI_ReachDRER_CPD
KPI_ReachDRR_CPD
KPI_ReachH_CPD
KPI_ReachHI_CPD
KPI_ReachHPM_CPD
KPI_ReachL_CPD
KPI_ReachLTSPD_CPD
KPI_ReachM_CPD
KPI_ReachRCRCEd_CPD
KPI_ReachS_CPD
KPI_ReachSI_CPD
KPI_ReachWASH_CPD
KPI_TrainFA_Tot
KPI_DonBlood_Tot
KPI_GB_Tot
KPI_PeopleVol_Tot
KPI_PStaff_Tot
KPI_noLocalUnits
KPI_IncomeLC_CHF
KPI_expenditureLC_CHF


Quick check for sum in Vol after imputing

In [17]:
fdrs_data_fdrs_na_columns[fdrs_data_fdrs_na_columns["KPI_Year"]==2021][["NSO_DON_name","KPI_Year","KPI_PStaff_Tot"]].sum()

NSO_DON_name      Austrian Red CrossBaphalali Eswatini Red Cross...
KPI_Year                                                     388032
KPI_PStaff_Tot                                             517843.0
dtype: object

In [18]:
fdrs_data_fdrs_na_columns[fdrs_data_fdrs_na_columns["KPI_Year"]==2020][["NSO_DON_name","KPI_Year","KPI_PStaff_Tot"]].sum()

NSO_DON_name      Afghan Red CrescentAlbanian Red CrossAlgerian ...
KPI_Year                                                     387840
KPI_PStaff_Tot                                             479118.0
dtype: object

### 2.3 Export imputed values in Excel

Quick process to ingest imputed values in an excel read by PBI

In [19]:
#generate kpi_code_ip names from kpi_code
kpi_code=kpi_reach_code+kpi_gov_fi_code
kpi_code_ip= [s + "_IP" for s in kpi_code]

In [20]:
#print original dataset 
fdrs_data_fdrs_na_columns

,KPI ID,KPI_DON_code,NSO_DON_name,NSO_ZON_name,iso_3,KPI_Year,KPI_DonBlood_Tot,KPI_GB_Tot,KPI_IncomeLC_CHF,KPI_PStaff_Tot,...,KPI_TrainFA_Tot,KPI_WasSubmitted,KPI_expenditureLC_CHF,KPI_noLocalUnits,validated,KPI_ReachRCRCEd_CPD,KPI_Climate_CPD,KPI_ClimateHeat_CPD,KPI_ReachHI_CPD,KPI_ReachHPM_CPD
0,DAF0012018,DAF001,Afghan Red Crescent,Asia Pacific,AFG,2018,0.0,15.0,1.803370e+07,2195.0,...,1.020000e+03,1.0,1.847249e+07,122.0,1.0,NaN,NaN,NaN,NaN,NaN
1,DAL0012018,DAL001,Albanian Red Cross,Europe and Central Asia,ALB,2018,6531.0,14.0,1.555610e+06,53.0,...,6.270900e+04,1.0,1.348786e+06,39.0,1.0,NaN,NaN,NaN,NaN,NaN
2,DDZ0012018,DDZ001,Algerian Red Crescent,Middle East and North Africa,DZA,2018,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,248.0,1.0,NaN,NaN,NaN,NaN,NaN
3,DUS0012018,DUS001,American Red Cross,Americas,USA,2018,2662986.0,15.0,3.656908e+09,19649.0,...,2.283480e+06,1.0,3.193028e+09,263.0,1.0,NaN,NaN,NaN,NaN,NaN
4,DAD0012018,DAD001,Andorran Red Cross,Europe and Central Asia,AND,2018,1084.0,9.0,1.177623e+06,20.0,...,2.901000e+03,1.0,1.379745e+06,1.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,DUA0012021,DUA001,Ukrainian Red Cross Society,Europe and Central Asia,UKR,2021,809.5,75.0,1.009907e+07,535.0,...,3.889000e+04,NaN,1.050011e+07,224.0,NaN,NaN,NaN,NaN,NaN,NaN
761,DUY0012021,DUY001,Uruguayan Red Cross,Americas,URY,2021,12.0,5.0,1.000076e+06,52.0,...,1.138667e+03,NaN,9.840340e+05,19.0,NaN,NaN,NaN,NaN,NaN,NaN
762,DVU0012021,DVU001,Vanuatu Red Cross Society,Asia Pacific,VUT,2021,0.0,10.0,7.895120e+05,37.0,...,6.026667e+02,NaN,9.459050e+05,7.0,NaN,NaN,NaN,NaN,NaN,NaN
763,DVE0012021,DVE001,Venezuelan Red Cross,Americas,VEN,2021,317.5,11.0,0.000000e+00,742.0,...,6.533000e+03,NaN,0.000000e+00,42.0,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
kpi_code_ip

['KPI_Climate_CPD_IP',
 'KPI_ClimateHeat_CPD_IP',
 'KPI_ReachCTP_CPD_IP',
 'KPI_ReachDRER_CPD_IP',
 'KPI_ReachDRR_CPD_IP',
 'KPI_ReachH_CPD_IP',
 'KPI_ReachHI_CPD_IP',
 'KPI_ReachHPM_CPD_IP',
 'KPI_ReachL_CPD_IP',
 'KPI_ReachLTSPD_CPD_IP',
 'KPI_ReachM_CPD_IP',
 'KPI_ReachRCRCEd_CPD_IP',
 'KPI_ReachS_CPD_IP',
 'KPI_ReachSI_CPD_IP',
 'KPI_ReachWASH_CPD_IP',
 'KPI_TrainFA_Tot_IP',
 'KPI_DonBlood_Tot_IP',
 'KPI_GB_Tot_IP',
 'KPI_PeopleVol_Tot_IP',
 'KPI_PStaff_Tot_IP',
 'KPI_noLocalUnits_IP',
 'KPI_IncomeLC_CHF_IP',
 'KPI_expenditureLC_CHF_IP']

In [22]:
#rename list of columns kpi_codes to kpi_codes_ip
fdrs_data_fdrs_na_columns.rename(columns=dict(zip(kpi_code, kpi_code_ip)), inplace=True)

In [23]:
#print dataset with new names
fdrs_data_fdrs_na_columns

,KPI ID,KPI_DON_code,NSO_DON_name,NSO_ZON_name,iso_3,KPI_Year,KPI_DonBlood_Tot_IP,KPI_GB_Tot_IP,KPI_IncomeLC_CHF_IP,KPI_PStaff_Tot_IP,...,KPI_TrainFA_Tot_IP,KPI_WasSubmitted,KPI_expenditureLC_CHF_IP,KPI_noLocalUnits_IP,validated,KPI_ReachRCRCEd_CPD_IP,KPI_Climate_CPD_IP,KPI_ClimateHeat_CPD_IP,KPI_ReachHI_CPD_IP,KPI_ReachHPM_CPD_IP
0,DAF0012018,DAF001,Afghan Red Crescent,Asia Pacific,AFG,2018,0.0,15.0,1.803370e+07,2195.0,...,1.020000e+03,1.0,1.847249e+07,122.0,1.0,NaN,NaN,NaN,NaN,NaN
1,DAL0012018,DAL001,Albanian Red Cross,Europe and Central Asia,ALB,2018,6531.0,14.0,1.555610e+06,53.0,...,6.270900e+04,1.0,1.348786e+06,39.0,1.0,NaN,NaN,NaN,NaN,NaN
2,DDZ0012018,DDZ001,Algerian Red Crescent,Middle East and North Africa,DZA,2018,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,248.0,1.0,NaN,NaN,NaN,NaN,NaN
3,DUS0012018,DUS001,American Red Cross,Americas,USA,2018,2662986.0,15.0,3.656908e+09,19649.0,...,2.283480e+06,1.0,3.193028e+09,263.0,1.0,NaN,NaN,NaN,NaN,NaN
4,DAD0012018,DAD001,Andorran Red Cross,Europe and Central Asia,AND,2018,1084.0,9.0,1.177623e+06,20.0,...,2.901000e+03,1.0,1.379745e+06,1.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,DUA0012021,DUA001,Ukrainian Red Cross Society,Europe and Central Asia,UKR,2021,809.5,75.0,1.009907e+07,535.0,...,3.889000e+04,NaN,1.050011e+07,224.0,NaN,NaN,NaN,NaN,NaN,NaN
761,DUY0012021,DUY001,Uruguayan Red Cross,Americas,URY,2021,12.0,5.0,1.000076e+06,52.0,...,1.138667e+03,NaN,9.840340e+05,19.0,NaN,NaN,NaN,NaN,NaN,NaN
762,DVU0012021,DVU001,Vanuatu Red Cross Society,Asia Pacific,VUT,2021,0.0,10.0,7.895120e+05,37.0,...,6.026667e+02,NaN,9.459050e+05,7.0,NaN,NaN,NaN,NaN,NaN,NaN
763,DVE0012021,DVE001,Venezuelan Red Cross,Americas,VEN,2021,317.5,11.0,0.000000e+00,742.0,...,6.533000e+03,NaN,0.000000e+00,42.0,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#export imputed values in an excel file. 
fdrs_data_fdrs_na_columns.to_excel(r"..\data\processed\1_FDRS_Wide_Format_IP_ALL.xlsx",index=False)

<a id="opti"></a>
## 3. Optimization: Get imputed values t-1 from backoffice to compare with imputed values t (optional)
---------------

In [ ]:
KPI = requests.get(f"https://data-api.ifrc.org/api/indicator?apiKey=21e401ae-6b35-404b-a72a-b74cce66dee3").json()
KPI = json_normalize(data=KPI)
kpi=(KPI.query('KPI_Note== "NS Reach - CPD"')["KPI_Code"]).to_list()
kpi_code=kpi+["KPI_TrainFA_Tot","KPI_DonBlood_Tot"]+["KPI_GB_Tot","KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_noLocalUnits","KPI_IncomeLC_CHF" ,"KPI_expenditureLC_CHF"]

In [ ]:
new_kpi=["KPI_Climate_CPD", "KPI_ClimateHeat_CPD", "KPI_ReachHI_CPD", "KPI_ReachHPM_CPD","KPI_ReachRCRCEd_CPD"]
for x in new_kpi:
    kpi_code.remove(x)

In [ ]:
kpi_code= [s + "_IP" for s in kpi_code]
kpi_float=kpi_code.copy()


Get imputed values from backoffice

In [ ]:
years=["2021"]

In [ ]:
time_series_imputed=api_function_imputed(years,kpi_code,kpi_float,api_key)
time_series_imputed

In [ ]:
time_series_imputed.drop("nan",axis=1,inplace=True)

In [ ]:
new_time_series_ip=pd.read_excel("FDRS2021NewKPIValues.xlsx")

In [ ]:
# renaming the column by index
new_time_series_ip.columns.values[2:7] =["KPI_ReachRCRCEd_CPD_IP", "KPI_Climate_CPD_IP","KPI_ClimateHeat_CPD_IP","KPI_ReachHI_CPD_IP","KPI_ReachHPM_CPD_IP"]

In [ ]:
time_series_imputed=time_series_imputed.merge(new_time_series_ip, how='left', on=['iso_3','KPI_Year'])
time_series_imputed

Code the values that are different.   
0 for values that do not need to be updated,   
1 for values that need to be updated. 

In [ ]:
KPI = requests.get(f"https://data-api.ifrc.org/api/indicator?apiKey=21e401ae-6b35-404b-a72a-b74cce66dee3").json()
KPI = json_normalize(data=KPI)
kpi=(KPI.query('KPI_Note== "NS Reach - CPD"')["KPI_Code"]).to_list()
kpi_code=kpi+["KPI_TrainFA_Tot","KPI_DonBlood_Tot"]+["KPI_GB_Tot","KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_noLocalUnits","KPI_IncomeLC_CHF" ,"KPI_expenditureLC_CHF"]

In [ ]:
#kpi_code=["KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_DonBlood_Tot","KPI_TrainFA_Tot","KPI_noLocalUnits","KPI_ReachDRER_CPD","KPI_ReachLTSPD_CPD","KPI_ReachDRR_CPD","KPI_ReachS_CPD","KPI_ReachL_CPD","KPI_ReachH_CPD","KPI_ReachWASH_CPD","KPI_ReachM_CPD","KPI_ReachCTP_CPD","KPI_ReachSI_CPD","KPI_IncomeLC_CHF","KPI_expenditureLC_CHF"]
#kpi_code=["KPI_PeopleVol_Tot"]

# Convert to type int to remove decimals from calculations
#fdrs_data_fdrs_na_columns[kpi_code]=fdrs_data_fdrs_na_columns[kpi_code].fillna(0).astype('int')
#fdrs_data_fdrs_na_columns[kpi_code]=fdrs_data_fdrs_na_columns[kpi_code].replace(0,np.nan)



In [ ]:
time_series_imputed

In [ ]:
fdrs_data_fdrs_na_columns

In [ ]:
kpi_code_imp= [s + "_IP" for s in kpi_code]
kpi_code_binary = [s + "_binary" for s in kpi_code]

In [ ]:
#kpi_code=["KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_DonBlood_Tot","KPI_TrainFA_Tot","KPI_noLocalUnits","KPI_ReachDRER_CPD","KPI_ReachLTSPD_CPD","KPI_ReachDRR_CPD","KPI_ReachS_CPD","KPI_ReachL_CPD","KPI_ReachH_CPD","KPI_ReachWASH_CPD","KPI_ReachM_CPD","KPI_ReachCTP_CPD","KPI_ReachSI_CPD","KPI_IncomeLC_CHF","KPI_expenditureLC_CHF"]



for kpi in list(kpi_code):
    kpi_code_imp=kpi+"_IP"
    kpi_code_binary=kpi+"_binary"
    fdrs_data_fdrs_na_columns.loc[time_series_imputed[kpi_code_imp] == fdrs_data_fdrs_na_columns[kpi],kpi_code_binary]=0 #0 if same
    fdrs_data_fdrs_na_columns.loc[time_series_imputed[kpi_code_imp] != fdrs_data_fdrs_na_columns[kpi],kpi_code_binary]=1 #1 if different
    
    fdrs_data_fdrs_na_columns.loc[fdrs_data_fdrs_na_columns[kpi].isna(),kpi_code_binary]=2
    #To be uncommented when the data is ready
    fdrs_data_fdrs_na_columns.loc[fdrs_data_fdrs_na_columns[kpi_code_binary]==2,kpi_code_binary]=0

            


In [ ]:
kpi_code_imp

In [ ]:
print(time_series_imputed[kpi_code_imp].dtypes)

In [ ]:
print(fdrs_data_fdrs_na_columns[kpi_code].dtypes)

In [ ]:
time_series_imputed.to_excel("C:/Users/simon.weiss/OneDrive - IFRC/Tech talk and files/1_PowerBI/1_FDRS/0.Data_Sources/1_FDRS_Wide_Format_IP_ALL_Staging.xlsx",index=False)

<a id="post"></a>
## 4. Post method - interact with Imp_Var? datapoint
---------------

2 methods: 
- Run first method if 3/ has not been generated. 
- Run second method if 3/ has been generated. 

### 4.0 General configuration

In [25]:
payload={}
files={}
headers = {}
#read private fdrs api key from config files
f = open(module_path+"/src/config/api_key.txt", "r")
api_key=f.readline()

**[User Input]** Select Years to imputed below

In [26]:
years=[2021]

**[User Input]** Variables to impute 

In [27]:
#manualy
#kpi_code=["KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_DonBlood_Tot","KPI_TrainFA_Tot","KPI_noLocalUnits","KPI_ReachDRER_CPD","KPI_ReachLTSPD_CPD","KPI_ReachDRR_CPD","KPI_ReachS_CPD","KPI_ReachL_CPD","KPI_ReachH_CPD","KPI_ReachWASH_CPD","KPI_ReachM_CPD","KPI_ReachCTP_CPD","KPI_ReachSI_CPD","KPI_IncomeLC_CHF","KPI_expenditureLC_CHF"]
#shorter
kpi_code=kpi_reach_code+kpi_gov_fi_code
#testing with Vol
kpi_code=["KPI_PStaff_Tot"]
kpi_code_ip= [s + "_IP" for s in kpi_code]


### 4.1 Post Method #1

In [33]:
for kpi in list(kpi_code_ip):
    for year in years:
        for kpi_don_code in list(set(fdrs_data_fdrs_na_columns["KPI_DON_code"])): 
            for label, content in (fdrs_data_fdrs_na_columns[kpi][(fdrs_data_fdrs_na_columns["KPI_Year"]==year)&(fdrs_data_fdrs_na_columns["KPI_DON_code"]==kpi_don_code)]).items():
                value = content
                if type(value)==float:
                    if not np.isnan(value):
                        value=np.int64(value)
                url = (f"https://data-api.ifrc.org/api/ImputedKPI?apiKey={api_key}&kpicode={kpi}&year={year}&don_code={kpi_don_code}&value={value}&user=simon.weiss@ifrc.org")
                time.sleep(2) # sleep for 2 seconds
                r = requests.post(url)
                print(url,r.text)

https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-4ac9-bfdb-516f3cd7e3c6&kpicode=KPI_PStaff_Tot_IP&year=2021&don_code=DIS001&value=116&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-4ac9-bfdb-516f3cd7e3c6&kpicode=KPI_PStaff_Tot_IP&year=2021&don_code=DHR001&value=1739&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-4ac9-bfdb-516f3cd7e3c6&kpicode=KPI_PStaff_Tot_IP&year=2021&don_code=DUZ001&value=650&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-4ac9-bfdb-516f3cd7e3c6&kpicode=KPI_PStaff_Tot_IP&year=2021&don_code=DCV001&value=119&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-4ac9-bfdb-516f3cd7e3c6&kpicode=KPI_PStaff_Tot_IP&year=2021&don_code=DYE001&value=454&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc

https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-4ac9-bfdb-516f3cd7e3c6&kpicode=KPI_PStaff_Tot_IP&year=2021&don_code=DSK001&value=419&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-4ac9-bfdb-516f3cd7e3c6&kpicode=KPI_PStaff_Tot_IP&year=2021&don_code=DMW001&value=260&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-4ac9-bfdb-516f3cd7e3c6&kpicode=KPI_PStaff_Tot_IP&year=2021&don_code=DSD001&value=721&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-4ac9-bfdb-516f3cd7e3c6&kpicode=KPI_PStaff_Tot_IP&year=2021&don_code=DDJ001&value=31&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-4ac9-bfdb-516f3cd7e3c6&kpicode=KPI_PStaff_Tot_IP&year=2021&don_code=DDE001&value=189787&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15f

https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-4ac9-bfdb-516f3cd7e3c6&kpicode=KPI_PStaff_Tot_IP&year=2021&don_code=DKP001&value=194&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-4ac9-bfdb-516f3cd7e3c6&kpicode=KPI_PStaff_Tot_IP&year=2021&don_code=DCD001&value=181&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-4ac9-bfdb-516f3cd7e3c6&kpicode=KPI_PStaff_Tot_IP&year=2021&don_code=DNL001&value=490&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-4ac9-bfdb-516f3cd7e3c6&kpicode=KPI_PStaff_Tot_IP&year=2021&don_code=DTG001&value=111&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-4ac9-bfdb-516f3cd7e3c6&kpicode=KPI_PStaff_Tot_IP&year=2021&don_code=DBI001&value=805&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=b08bc654-15fc-

ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

### 4.2 Post methods #2 (only if 3.used)

In [ ]:
#fdrs_data_fdrs_na_columns_test.fillna(0,inplace=True)
for kpi in list(kpi_code):
    kpi_binary = kpi + "_binary"
    for year in years:
        for kpi_don_code in list(set(fdrs_data_fdrs_na_columns["KPI_DON_code"])): 
            for bi in (fdrs_data_fdrs_na_columns[kpi_binary][(fdrs_data_fdrs_na_columns["KPI_Year"]==year)&(fdrs_data_fdrs_na_columns["KPI_DON_code"]==kpi_don_code)]):
                binary = bi
                if binary == 1.0: 
                    for label, content in (fdrs_data_fdrs_na_columns[kpi][(fdrs_data_fdrs_na_columns["KPI_Year"]==year)&(fdrs_data_fdrs_na_columns["KPI_DON_code"]==kpi_don_code)]).items():
                        value = content
                        if type(value)==float:
                            if not np.isnan(value):
                                value=np.int64(value)
                                value = value
                        #if value == "nan":
                        if np.isnan(value):
                            value = ""
                        kpi_post=kpi+"_IP"
                        url = (f"https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey={api_key}&kpicode={kpi_post}&year={year}&don_code={kpi_don_code}&value={value}&user=simon.weiss@ifrc.org")
                        r = requests.post(url,headers=headers, data=payload, files=files)
                        #print(url,r.text)
                        #print(r.request.body)
                        #print(r.request.headers)

                        

### 4.3 Final update lattest update time excel

In [ ]:
df1 = pd.DataFrame([datetime.now()], columns=['Lattest Update Date'])
df1.to_excel(module_path+"data/processed/lattest_update.xlsx", index=False)


Time to execute all : before  optimization : 111min   
After optimization : 2min